# LDA에서 Dynamic Topic Model

<img src="figs/dtm_fig.png">

* LDA를 time slice 단위로 여러개를 붙인 형태
* 토픽 분포에 해당하는 파라미터인 Beta값을 time slice가 넘어갈때 넘겨주는 것으로 시간대별 토픽 분포를 반영하게 함

<img src="figs/dtm_fig2.png">

### 1. 대화내용 불러오기

In [1]:
import pandas as pd
import numpy as np
import pickle
from pprint import pprint
import re

In [2]:
with open("data/cleaned_data.pk", "rb") as f:
    data = pickle.load(f)

data.reset_index(drop=True, inplace=True)
print(data.head())
print(data.info())

               Date User            Message
0  2017/01/01 20:32   무지  이거보면 왜 갓창정인지 알게된다
1  2017/01/01 21:40  어피치                창정헌
2  2017/01/01 22:19  어피치   라이언은 내일부터 연구실 출근
3  2017/01/01 22:20   무지          파티 하는거 아님
4  2017/01/01 22:39  프로도                헬파티
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162861 entries, 0 to 162860
Data columns (total 3 columns):
Date       162861 non-null object
User       162861 non-null object
Message    162861 non-null object
dtypes: object(3)
memory usage: 3.7+ MB
None


### 2. 분석시기 설정하기

In [3]:
# 시간정보 열 datetime 정보로 변환
data['Date'] = pd.to_datetime(data['Date'])
# 인덱스 넣기
data = data.set_index('Date')
data.head()

,User,Message
Date,,
2017-01-01 20:32:00,무지,이거보면 왜 갓창정인지 알게된다
2017-01-01 21:40:00,어피치,창정헌
2017-01-01 22:19:00,어피치,라이언은 내일부터 연구실 출근
2017-01-01 22:20:00,무지,파티 하는거 아님
2017-01-01 22:39:00,프로도,헬파티


#### 연도별 대화내용 분리 (DTM 분석을 위해)

In [4]:
# 지정된 시기별로 분리
year2017 = data['2017-01-01' : '2017-12-31']
year2018 = data['2018-01-01' : '2018-12-31']
year2019 = data['2019-01-01' : '2019-12-31']

In [5]:
print(year2017.head())
print(year2018.head())
print(year2019.head())

                    User            Message
Date                                       
2017-01-01 20:32:00   무지  이거보면 왜 갓창정인지 알게된다
2017-01-01 21:40:00  어피치                창정헌
2017-01-01 22:19:00  어피치   라이언은 내일부터 연구실 출근
2017-01-01 22:20:00   무지          파티 하는거 아님
2017-01-01 22:39:00  프로도                헬파티
                    User             Message
Date                                        
2018-01-01 00:04:00   무지         새해복 마니받으시게나
2018-01-01 00:05:00  어피치        새해복 많이 받으시오들
2018-01-01 00:06:00   튜브                 들어오
2018-01-01 00:07:00   튜브  12시 00에 딱 초코의몽을 삿다
2018-01-01 00:07:00   무지                   곧
                    User    Message
Date                               
2019-01-01 00:02:00  프로도  쉬발럼들 사랑한다
2019-01-01 00:02:00   무지       사랑한다
2019-01-01 00:02:00   무지      내 친구들
2019-01-01 00:02:00   무지        고맙다
2019-01-01 00:02:00   무지  새해도 잘 부탁해


In [6]:
print(year2017.info())
print(year2018.info())
print(year2019.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 92998 entries, 2017-01-01 20:32:00 to 2017-12-31 23:41:00
Data columns (total 2 columns):
User       92998 non-null object
Message    92998 non-null object
dtypes: object(2)
memory usage: 2.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 52570 entries, 2018-01-01 00:04:00 to 2018-12-31 23:54:00
Data columns (total 2 columns):
User       52570 non-null object
Message    52570 non-null object
dtypes: object(2)
memory usage: 1.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17293 entries, 2019-01-01 00:02:00 to 2019-10-14 19:44:00
Data columns (total 2 columns):
User       17293 non-null object
Message    17293 non-null object
dtypes: object(2)
memory usage: 405.3+ KB
None


##### 시기별로 나눠서 데이터 저장하기

In [7]:
# time_slice별로 데이터 갯수를 저장할 리스트 생성
time_slice = []

# 시기별 데이터 가져오기
slice0 = list(year2017['Message'])
slice1 = list(year2018['Message'])
slice2 = list(year2019['Message'])

# LDA 분석할 때와 비슷하게 모든 분석 대상 텍스트를 담는 리스트 생성
tokenized_data = [msg.split() for msg in (slice0 + slice1 + slice2)]

# 각 slice에 들어 있는 갯수를 원소로 갖는 리스트 생성
time_slice.append(len(slice0))
time_slice.append(len(slice1))
time_slice.append(len(slice2))

In [8]:
print(len(tokenized_data))
print(time_slice)
print(slice0[:5])
print(slice1[:5])
print(slice2[:5])
pprint(tokenized_data[:5])

162861
[92998, 52570, 17293]
['이거보면 왜 갓창정인지 알게된다', '창정헌', '라이언은 내일부터 연구실 출근', '파티 하는거 아님', '헬파티']
['새해복 마니받으시게나', '새해복 많이 받으시오들', '들어오', '12시 00에 딱 초코의몽을 삿다', '곧']
['쉬발럼들 사랑한다', '사랑한다', '내 친구들', '고맙다', '새해도 잘 부탁해']
[['이거보면', '왜', '갓창정인지', '알게된다'],
 ['창정헌'],
 ['라이언은', '내일부터', '연구실', '출근'],
 ['파티', '하는거', '아님'],
 ['헬파티']]


### 3. Dynamic Topic Model 돌리기

In [9]:
from gensim.models import ldaseqmodel
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary, bleicorpus
from gensim.matutils import hellinger
from gensim import corpora
from tqdm import tqdm_notebook
from time import time

import os

#### dictionary와 corpus 만들기

In [10]:
# DTM dictionary 저장.
if not os.path.exists('kakao(DTM)_dict'):
    dictionary = corpora.Dictionary(tokenized_data)
    dictionary.save('kakao(DTM)_dict')
    print(dictionary)
else:
    dictionary = Dictionary.load('kakao(DTM)_dict')

# DTM Corpus 저장.
if not os.path.exists('kakao(DTM)_corpus'):
    corpus = [dictionary.doc2bow(doc) for doc in tokenized_data]
    corpora.BleiCorpus.serialize('kakao(DTM)_corpus', corpus)
else:
    corpus = bleicorpus.BleiCorpus('kakao(DTM)_corpus')

### Run DTM model 

In [13]:
# 주의! 매우매우매우 오래걸릴지도 모릅니다... 시간과 여유가 있을때 해보세요!
NUM_TOPICS = 4

if not os.path.exists('kakao(DTM)_model'):
    dtm_model = ldaseqmodel.LdaSeqModel(corpus=corpus, id2word=dictionary, time_slice=time_slice, num_topics=NUM_TOPICS, passes=10)
    dtm_model.save('kakao(DTM)_model')
else:
    dtm_model = ldaseqmodel.LdaSeqModel.load('kakao(DTM)_model')

##### DTM 결과 보기

In [15]:
# 고정된 시간 내에서 전체 토픽보기
pprint(dtm_model.print_topics(time=1, top_terms=10)) 

[[('굿', 0.05831850538739241),
  ('지금', 0.012331524778776757),
  ('오', 0.0122166677634343),
  ('역시', 0.010455539788893783),
  ('내일', 0.009190718899343277),
  ('이미', 0.008990207437333388),
  ('왜', 0.007369435495622992),
  ('낼', 0.005757033715288104),
  ('야', 0.0051241059091660665),
  ('아직', 0.004872538898427595)],
 [('핳', 0.03620256743725127),
  ('그럼', 0.017891407390810514),
  ('나', 0.014458865122934468),
  ('나도', 0.01339997787202228),
  ('걍', 0.012045925968074068),
  ('닥쳐', 0.011429731540920987),
  ('나는', 0.009952254204843707),
  ('핳핳', 0.009887808067018595),
  ('뭐', 0.009682205321122319),
  ('Search', 0.008825802051635289)],
 [('아', 0.02210906965432367),
  ('난', 0.011473428499934674),
  ('음', 0.009178877503758395),
  ('오늘', 0.008736621924946206),
  ('일단', 0.007706724944217089),
  ('다', 0.006729730613141478),
  ('그냥', 0.006459893475380204),
  ('솬', 0.004414683249180747),
  ('흑', 0.00408539709200058),
  ('너무', 0.003937975344398072)],
 [('프로도', 0.018715752188847568),
  ('근데', 0.0173516806